In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
# (i, j, value)
error_diffusion_masks = {
    "Floyd Steinberg": [
        (0,  1, 7/16),
        (1, -1, 3/16),
        (1,  0, 5/16),
        (1,  1, 1/16),
    ],

    "Stevenson Arce": [
        (0,  2, 32/200),
        (1, -2, 12/200), (1, -1, 26/200), (1,  0, 30/200), (1,  1, 16/200), (1,  2, 12/200),
        (2, -2,  5/200), (2, -1, 12/200), (2,  0, 12/200), (2,  1,  5/200),
    ],

    "Burkes": [
        (0,  0,  8/32),
        (0,  1,  4/32),
        (1, -2,  2/32), (1, -1,  4/32), (1,  0,  8/32), (1,  1,  4/32), (1,  2,  2/32),
    ],

    "Sierra": [
        (0,  0,  5/32),
        (0,  1,  3/32),
        (1, -2,  2/32), (1, -1,  4/32), (1,  0,  5/32), (1,  1,  4/32), (1,  2,  2/32),
        (2, -1,  2/32), (2,  0,  3/32), (2,  1,  2/32),
    ],

    "Stucki": [
        (0,  0,  8/42),
        (0,  1,  4/42),
        (1, -2,  2/42), (1, -1,  4/42), (1,  0,  8/42), (1,  1,  4/42), (1,  2,  2/42),
        (2, -2,  1/42), (2, -1,  2/42), (2,  0,  4/42), (2,  1,  2/42), (2,  2,  1/42),
    ],

    "Jarvis Judice Ninke": [
        (0,  0,  7/48),
        (0,  1,  5/48),
        (1, -2,  3/48), (1, -1,  5/48), (1,  0,  7/48), (1,  1,  5/48), (1,  2,  3/48),
        (2, -2,  1/48), (2, -1,  3/48), (2,  0,  5/48), (2,  1,  3/48), (2,  2,  1/48),
    ],
}


## Primeira implementação: Usando a varredura de cima a baixo e sempre da esquerda para a direita

In [ ]:
def apply_dithering_unidirectional(image_path, mask):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = image.astype(np.float64)
    height, width = image.shape
    out_image = np.zeros_like(image, dtype=np.uint8)
    for i in range(height):
        for j in range(width):
            out_image[i][j] = 0 if image[i][j] < 128 else 255
            error = image[i][j] - out_image[i][j]
            for i_rel, j_rel, value in mask:
                new_i = i + i_rel
                new_j = j + j_rel
                if (0<= new_i < height and 0<= new_j < width):
                    image[new_i, new_j] = image[new_i, new_j] + error * value
    return out_image

def apply_serpentine_scanning(image_path, mask):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = image.astype(np.float64)
    height, width = image.shape
    out_image = np.zeros_like(image, dtype=np.uint8)
    for i in range(height):
        if (i % 2 == 0):
            column_range = range(width)
        else:
            column_range = range(width - 1, -1, -1)
        for j in column_range:
            out_image[i][j] = 0 if image[i][j] < 128 else 255
            error = image[i][j] - out_image[i][j]
            for i_rel, j_rel, value in mask:
                if (i % 2 != 0):
                    j_rel = -j_rel
                new_i = i + i_rel
                new_j = j + j_rel
                if (0<= new_i < height and 0<= new_j < width):
                    image[new_i, new_j] = image[new_i, new_j] + error * value
    return out_image
for mask_name, mask_value in error_diffusion_masks.items():
    original = cv2.imread('../imagens/baboon_monocromatica.png', cv2.IMREAD_GRAYSCALE)
    unidirectional = apply_dithering_unidirectional('../imagens/baboon_monocromatica.png', mask_value)
    serpentine = apply_serpentine_scanning('../imagens/baboon_monocromatica.png', mask_value)
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))

    axes[0].imshow(original, cmap='gray')
    axes[0].set_title("Imagem original")
    axes[0].axis('off')

    axes[1].imshow(unidirectional, cmap='gray')
    axes[1].set_title( f'Algoritimo: Unidirection, Mascara: {mask_name}')
    axes[1].axis('off')

    axes[2].imshow(serpentine, cmap='gray')
    axes[2].set_title(f'Algoritimo: Serpentine, Mascara: {mask_name}')
    axes[2].axis('off')
    plt.tight_layout()
    plt.show()
    